In [1]:
import numpy as np
import pandas as pd
import sys
import datetime
from datagenerator_creation import DataGenerator
from build_model import build_model
from keras.callbacks import Callback
from keras.models import load_model
from utils import getLettersAndDiacritics, convert_to_two_hot

Using TensorFlow backend.


42


In [2]:
def split_data(df, split_at=.3):
    m = len(df)
    split_point = int((1-split_at)*m)

    train = "".join(df['Bayt_Text'][:split_point])
    val = "".join(df['Bayt_Text'][split_point:])

    return train, val

In [3]:
path = 'data/cleaned_arabic_dataset.csv'
df = pd.read_csv(path)
train, val = split_data(df[:1000])

train_generator = DataGenerator(train)
val_generator = DataGenerator(val)


In [4]:
model = build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 128)           87552     
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 42)                10794     
Total params: 492,586
Trainable params: 492,586
Non-trainable params: 0
_________________________________________________________________


In [5]:
# print('Loading Weights...')
# weights_path = 'weights/my_model_weights2.h5'
# model = load_weights(model, weights_path)

In [7]:
model.fit_generator(generator=train_generator,
                validation_data=val_generator,
                use_multiprocessing=True,
                workers=3,
                epochs=5,
                verbose=1)

Epoch 1/5
216/216 [==============================] - 59s 273ms/step - loss: 4.2149 - val_loss: 4.1180
Epoch 2/5
216/216 [==============================] - 50s 232ms/step - loss: 4.0291 - val_loss: 3.8363
Epoch 3/5
216/216 [==============================] - 45s 210ms/step - loss: 3.8249 - val_loss: 3.7261
Epoch 4/5
216/216 [==============================] - 45s 208ms/step - loss: 3.7056 - val_loss: 3.6016
Epoch 5/5
216/216 [==============================] - 45s 209ms/step - loss: 3.5730 - val_loss: 3.5629


In [7]:
# model = load_model('my_model')

In [12]:
def convert_vec_to_char(two_hot_vec):
    indexes = np.where(two_hot_vec==1)[0]
    return index_to_letter[indexes[0]] + index_to_letter[indexes[1]]

def convert_indexes_to_char(indexes):
    return index_to_letter[indexes[0]] + index_to_letter[indexes[1]]

In [13]:
letters, diacritics = getLettersAndDiacritics()
# perhabs there is a problem here with the arrangement
lettersAndDiacritics = letters + diacritics
index_to_letter = dict((i, c) for i, c in enumerate(lettersAndDiacritics))

In [39]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(100, preds, 1)/100 
    out1 = np.random.choice(range(42),1, p=probas.ravel())
    out2 = np.random.choice(range(42),1, p=probas.ravel())
    dia = [0, 38, 39, 40, 41, 42]
    while(out1==out2 or (out1 in dia  and out2 in dia)):
        out2 = np.random.choice(range(42),1, p=probas.ravel())

    return [out1[0], out2[0]]

In [40]:
def generate_output(model, Tx=40, n_x=42):
    generated = ''
    # sentence = text[start_index: start_index + Tx]
    # sentence = '0'*Tx
    # usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    usr_input = 'بلب'

    sentence = ('{0:0<' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input

    sys.stdout.write("\n\nHere is your poem: \n\n")
    sys.stdout.write(usr_input)
    for i in range(100):
        x_pred = np.zeros((1, Tx, n_x))
        two_hot_vec = np.squeeze(convert_to_two_hot(sentence.split('0')[0]))
        x_pred[0][:len(two_hot_vec)] = two_hot_vec
        preds = model.predict(x_pred, verbose=0)[0]
#         print(x_pred)
#         print(np.argmax(preds))

        next_index = sample(preds, temperature=1.0)
#         print(next_index)
        next_char = convert_indexes_to_char(next_index)
        generated += next_char
        sentence = sentence + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
        if next_char == '\n':
            continue

In [42]:
generate_output(model)



Here is your poem: 

بلب بدَْجه ُاْرْىسع كْوَبَْاَْْاِْذْاملْاََْاَْا ْىَنجْْنِبَْر تْاْعْْع ىْاش ُاَْْاْاَيَنَُْتَْْو َْى يَْ َ َْب املْامْايِاَْ اَنْاظْْو َْىْررِ لَْْنْذْقُآْنْروَْاْحتَََُْاُْرَََْْْئْْرلبءْنَْاْا رْهْدَْ